In [14]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;


In [3]:
using Pkg;
Pkg.instantiate()

# Задача теплопроводности

$\hspace{0.25cm}$ Во многих инженерных задачах важным аспектом явлется знание распределения температуры в теле. 
Всем известно, что при нагреве, тела расширяются и это приводит к возникновению тепловых напряжений. Если мы знаем количество подводимого и выделяемого телом тепла, то можно определенить распределение температуры в его объеме, а следовательно - распределение напряжений, знание которых необходимо при проектировании вращающихся механизмов. Например, реактивных двигателей, паровых генераторов и даже червячных редукторов.

## Уравнения переноса тепла
$\hspace{0.25cm}$ Перенос тепла или теплопередача является сложным процессом, который можно разделить на три элементарных способа переноса теплоты: теплопроводность, конвекция и тепловое излучение. Теплопроводностью называется процесс перенос теплоты при непосредственном контакте более нагретых элементов тела (или среды) с менее нагретыми, осуществляемый посредством хаотического движения и взаимодействия микрочастиц (молекул, атомов, электронов, ионов). Явление конвекции наблюдается в движущихся жидкостях или газах. Перенос теплоты при этом происходит за счет перемещения вещества в пространстве. Тепловым излучением называется явление переноса теплоты в виде электромагнитных волн с двойным взаимным превращением - тепловой энергии в лучистую и обратно. В большинстве случаев виды теплообмена не существуют обособленно. Например, обмен теплотой между твердой поверхностью и жидкостью (или газом) происходит путем теплопроводности и конвекции одновременно и называется конвективным теплообменом или теплоотдачей.
Процесс теплопроводности неразрывно связан с распределением температуры внутри тела. Поэтому при его изучении прежде всего необходимо установить некоторые понятия:

$\hspace{0.25cm}$ Температурное поле – это совокупность значений температуры для всех точек пространства в данный момент времени.

$\hspace{0.25cm}$ Температурный градиент – вектор, направленным по нормали к изотермической поверхности в сторону возрастания температуры, °С/м.

$\hspace{0.25cm}$ Тепловой поток. Теплота самопроизвольно переносится только в сторону убывания температуры. Количество теплоты, переносимое через какую-либо изотермическую поверхность в единицу времени, называется тепловым потоком Q. Тепловой поток, отнесенный к единице площади изотермической поверхности, называется плотностью теплового потока q. Плотность теплового потока есть вектор, направление которого совпадает с направлением распространения теплоты в данной точке и противоположно направлению вектора температурного градиента.

$\hspace{0.25cm}$ Изучая процесс теплопроводности в твердых телах, Фурье экспериментально установил, что количество переданной теплоты пропорционально падению температуры, времени и площади сечения, перпендикулярного направлению распространения теплоты. Если количество переданной теплоты отнести к единице площади сечения и единице времени, то установленную зависимость можно записать закон Фурье:
$$

- q \mathbf{n} = -\lambda \nabla \hspace{0.1cm}T,

$$
где $q$ - плотность теплового потока, [$\frac{Вт}{м^2}$], $\lambda$ - коэффициент теплопроводности, [$\frac{Вт}{м \cdot К}$]. Является физическим параметром, характеризует способность данного вещества проводить тепло и численно равен количеству теплоты, переданному в единицу времени через единицу поверхности при градиенте температуры, равном единице. Наилучшей теплопроводностью из металлов обладают серебро - 400 [$\frac{Вт}{м \cdot К}$], медь - 395 [$\frac{Вт}{м \cdot К}$]. Алюминий распространен как материал для изготовления радиаторов охлаждения и имеет коэффициент теплопроводности 210 [$\frac{Вт}{м \cdot К}$], железо - 92 [$\frac{Вт}{м \cdot К}$]. Примеси значительно снижают коэффициент теплопроводности, так как структурные неоднородности сильно рассеивают электроны. Например, у меди с примесями мышьяка $\lambda$ = 142 [$\frac{Вт}{м \cdot К}$]. Для сплавов, в отличие от чистых металлов, коэффициент теплопроводности увеличивается с ростом температуры. В целом коэффициент теплопроводности убывает с уменьшением плотности. Газы имеют теплопроводность в разы ниже, например воздух - 0,026 [$\frac{Вт}{м \cdot К}$], аргон - 0,016 [$\frac{Вт}{м \cdot К}$]. В общем случае коэффицент теплопроводности не является константой и зависит от  рода вещества и температуры. 
 
$\hspace{0.25cm}$Еще одним важным уравнением является дифференциальное уравнение энергии:
$$
\rho \cdot C_m \cdot \frac{\partial T}{\partial t}=q_v - div\hspace{0.1cm}q,
$$
где $\rho$ - плотность вещества, $\frac{кг}{м^3}$, $C_m$ - весовая теплоемкость вещества, $\frac{Дж}{кг\cdot К}$
$q_v$ - мощность внутреннего источника теплоты, $\frac{Вт}{м^3}$. Данное уравнение говорит о том, что температура изменяется из-за выноса тепла или из-за внутреннего источника теплоты.

$\hspace{0.25cm}$Если подставить уравнение Фурье в уравнение энергии то получим уравнение Фурье-Кирхгофа:
$$
\rho \cdot C_m \cdot \frac{\partial T}{\partial t}=q_v + div(\lambda\hspace{0.1cm}grad\hspace{0.1cm}T).
$$

$\hspace{0.25cm}$Для решения задач так же необходимы условия однозначности: геометрические условия, временные (стационарный или нестационарный режим), физические условия (функциональное или числовое задание всех характеристик входящих в задачу), а также граничные условия.
Виды граничных условий: 

1) I-го рода или условие Дирихле: задание числового значение распределение температуры на поверхности тела для каждого момента времени.
$$
    T(\mathbf{x}) = T_{D}(\mathbf{x}),\text{ для } \mathbf{x} \in {\partial {\Omega}_D}
$$

где $ \partial {\Omega}_D $ - граница области, на которой заданы условия Дерихле, а $T_D$ - температура на данной границе.

2) II-го рода или условие Неймана: задание первой производной на границе (задается величина теплового потока для каждой точки поверхности тела)

$$
   q_s=-\lambda\frac{dT}{dx}
$$
3) III-го рода: задается температура окружающей среды и закон теплообмена между поверхностью тела и окружающей средой в процессе охлаждения и нагревания. Для его описания используется закон Ньютона – Рихмана (правая часть уравнения): плотность теплового потока пропорциональна разности температур поверхности тела. 
$$
   -\lambda\frac{dT}{dn}=\alpha (T_с-T_o),
$$
где $\alpha$ - коэффициент теплоотдачи, характеризующий интенсивность теплообмена между поверхностью тела и окружающей средой, [$\frac{Вт}{м^2 \cdot К}$], $T_с$ и $T_o$ - температура стенки и окружающей среды соответственно, [K].

$\hspace{0.25cm}$ Коэффициент теплоотдачи при естественной конвекции для газа 5-15 [$\frac{Вт}{м^2 \cdot К}$], для жидкости 100-1000 [$\frac{Вт}{м^2 \cdot К}$]. При вынужденной конвекции 10-500 [$\frac{Вт}{м^2 \cdot К}$] для газа и 500-10000 [$\frac{Вт}{м^2 \cdot К}$] для воды.

## Слабая постановка
$\hspace{0.25cm}$ Мультифизическое моделирование основано на дифференциальных уравнениях в частных производных (ДУЧП). Эти ДУЧП обычно выводятся из законов сохранения физических величин, таких как законы сохранения массы, энергии и импульса. Эти хорошо известные законы сохранения могут, без каких-либо дополнительных предположений, быть сформулированы в виде интегральных уравнений для произвольных областей.

$\hspace{0.25cm}$ Члены уравнений с объёмными интегралами описывают то, что хранится внутри доменов или добавляется источниками, в то время как члены с поверхностными интегралами описывают взаимодействие с соседними доменами или внешней средой. При условии, что все задействованные функции достаточно гладкие, по теореме Остроградского-Гаусса можно преобразовать поверхностные интегралы в объемные интегралы. Поскольку расчётная область является произвольной, результирующее уравнение должно быть действительным или определённым в каждой точке по отдельности, что в итоге сводит интегральное уравнение в ДУЧП.

$\hspace{0.25cm}$ Однако в этой стандартной процедуре вывода ДУЧП есть небольшой недостаток: задействованные функции (в частности, свойства материала) не всегда достаточно гладкие, чтобы оправдать повсеместное применение теоремы Остроградского-Гаусса. Таким образом, итоговое ДУЧП является слишком строгим в том смысле, что он не допускает всех физически разумных решений (и именно здесь "вступает в игру" слабая форма). Слабое уравнение частично изменяет процедуру вывода, возвращая интегральную формулировку, которая является менее строгой, чем ДУЧП. Это означает, что уравнения в слабой форме на самом деле ближе к лежащей в основе явления физике, чем классические уравнения в частных производных.

$\hspace{0.25cm}$ Проиллюстрируем типовую процедуру получения его слабого аналога. С этой целью давайте рассмотрим уравнение :

$$

- \nabla \cdot (-с \nabla u)=f

$$
где $c$ и $f$ - являются коэффициентом теплопроводности и потоком теплоты соответственно. 

$\hspace{0.25cm}$ Если неизвестная функция $u$ является решением исходного ДУЧП, она также является решением следующего интегрального уравнения:
$$
\int_{\Omega}^{}\nabla \cdot (-с \nabla u) v=\int_{\Omega}^{}fv
$$
$\hspace{0.25cm}$Это результат умножения ДУЧП на произвольную функцию $v$ с последующим применением интеграла по $Omega$ с обеих сторон уравнения.
В свою очередь функция $u$ , которая разрешает интегральное уравнение для достаточного числа различных функций $v$, является отличным кандидатом для решения исходного ДУЧП. В итоге функции $v$ называются пробными (или тестовыми) функциями и играют ключевую роль в методе конечных элементов. 

$\hspace{0.25cm}$ Преобразование ДУЧП в это интегральное уравнение уже отражает природу слабой формы, но обычно за этим следует второй шаг. На этом шаге применяется нтегрирование по частям для производных второго порядка. В нашем случае мы можем переписать левую часть предыдущего интегрального уравнения следующим образом:
$$
\int_{\Omega}^{}\nabla \cdot (-с \nabla u) v=\int_{\Omega}^{}c\nabla u \cdot \nabla v + \int_{\partial {\Omega}}^{}-c \nabla u \cdot nv
$$
где $\partial {\Omega}$ является границей области $\Omega$ и $n$ – вектор нормали, ориентированный в сторону от области. Используя эту формулу, итоговая слабая форма уравнения записывается как:
$$
\int_{\Omega}^{}c\nabla u \cdot \nabla v - \int_{\Omega}^{}fv + \int_{\partial {\Omega}}^{}-c \nabla u \cdot nv = 0
$$
$\hspace{0.25cm}$ Интеграл по границе отвечает за взаимодействие с окружающим область  пространством(описывает тепловой поток через границу.Если теплового потока нет, то граница теплоизолирована).

$\hspace{0.25cm}$ Функция $u$ называется слабым решением ДУЧП, если она является решением слабого уравнения для множества различных пробных функций, $v$. Точнее, мы рассматриваем не просто одну пробную функцию. $v$ представляет целый класс пробных функций, каждая из которых соответствует уравнению. Конечно, мы никогда не сможем рассмотреть все возможные пробные функции, так как это привело бы к бесконечному количеству уравнений. На практике нам нужно конечное число хорошо подобранных пробных функций. Именно в этот момент "в игру" вступают конечные элементы.

$\hspace{0.25cm}$ Для вычислительной области перед запуском вашего исследования должна быть построена так называемая сетка. Сетка делит геометрию на набор меньших объёмов, которые называют конечными элементами. Затем пробные функции могут быть определены с использованием многочленов (полиномов) для каждого элемента таким образом, чтобы они отличались от нуля только для небольшой группы соседних элементов и были равны нулю вне группы. Наиболее распространенный тип многочленов, построенных таким образом, известен как функции формы Лагранжа.
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
## Пример решения задачи
Рассмотрим стационарный случай, тогда уравнение Фурье-Кирхгофа для теплопроводности в сплошной среде имеет вид 
$$
% \label{eq_eigen} \tag{1}
- \nabla \cdot \mathbf{\lambda} \nabla T(\mathbf{x}) + q_v(\mathbf{x}) = 0, \text{ для } \mathbf{x}\in \Omega
$$

где $\mathbf{x} = \{x \hspace{0.1cm} y \hspace{0.1cm} z \}^T$ - вектор пространственных координат, $ \Omega $ - область решения задачи, определяема геометрией рассматриваемого тела,  $\mathbf{\lambda} = \{\lambda_{xx} \hspace{0.1cm} \lambda_{yy} \hspace{0.1cm} \lambda_{zz} \}^T$ - вектор коэффициентов теплопроводности в направлениях $ x $, $ y $ и $ z $, размерности $кВт / (м \cdot K)$, и $q_v$ - мощность внутреннего источника теплоты, размерности $кВт / м^3$. В уравнении неизвестным и является скалярное поле температур $T(\mathbf{x})$ и именно его неободимо определить при решении задачи.

Кроме определяющего уравнения обычно присутствуют дополнительные ограничения, задающие температуру или тепловой поток на поверхности тела, являющиеся граничными условиями для нашей задачи. Первый тип ограничений называют граничными условиями I-го рода, или условиям Дерихле

$$
    T(\mathbf{x}) = T_{D}(\mathbf{x}),\text{ для } \mathbf{x} \in {\partial {\Omega}_D}
$$

где $ \partial {\Omega}_D $ - граница области, на которой заданы условия Дерихле, а $T_D$ - температура на данной границе.

Второй тип ограничений называют граничными условиями II-го рода, или условиями Неймана
 №№№№№№№ (2го  рода это задание первой производной а именно плотности теплового потока q_s= $\lambda$ dT/dx вроде как потерялась лямбда в формуле ниже)

$$
    \nabla T(\mathbf{x}) \cdot \mathbf{n}(\mathbf{x}) = Q(\mathbf{x}), \text{ для } \mathbf{x}\in{\partial {\Omega}_N}
$$
где $ \partial {\Omega}_N $ - граница области, на которой заданы условия Неймана, $\mathbf{n}(\mathbf{x})$ - вектор нормали к границе, а $Q(\mathbf{x})$ - тепловой поток на данной границе.

## Формулировка задачи

В данной задаче необходимо рассчитать температурное поле радиатора охлаждения, подключенного к однофазному диодному мосту в общем корпусе. 

Задача будет рассмотрена в двухмерной постановке.

## Создание геометрии


Изначально родным для Gridap форматом геометрии является формат .json, имеющий определенную стркутуру. Однако более удобно использовать существующую библотеку gmsh для построения геометрии, имеющую большое количество возможностей и являющейся, по мнению авторов, наиболее мощной open-source библиотекой для построения расчетных неструктурированых сеток.


In [15]:
using Gmsh

# инициализация gmsh
gmsh.initialize() # инициализация ядра gmsh

gmsh.option.setNumber("General.Terminal", 0); # отключение вывода в терминал логов

ms_max = 1;
ms_min = 0.1;
MSFC = 3;
# порядок элементов
EL_ORDER = 1


#установка параметров gmsh

# gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", MSFC);
# gmsh.option.setNumber("Mesh.MeshSizeMax", diam/2);
# gmsh.option.setNumber("Mesh.MeshSizeMin", diam/5);
gmsh.option.setNumber("Mesh.MaxNumThreads3D", 1);
gmsh.option.setNumber("Mesh.ElementOrder", EL_ORDER);
# gmsh.option.setNumber("Mesh.Algorithm3D", 9);
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2);

# создание модели
gmsh.model.add("t4")

# параметры радиатора
height = 5;
width = 10;
rib_th = 1;
rib_num = 3;

# зазор между ребрами
sp = (width - rib_num * rib_th) / (rib_num - 1);

# размеро элементов
Lc1 = 0.3;


# создание "фабрики"
factory = gmsh.model.geo;

# создание угловых узлов
factory.addPoint(-width/2, 0,       0, Lc1, 1);
factory.addPoint( width/2, 0,       0, Lc1, 2);
factory.addPoint( width/2, height,  0, Lc1, 3);
factory.addPoint(-width/2, height,  0, Lc1, 4);

# создание угловых узлов ребер
x1 = -width/2 + rib_th;
x2 = -width/2 + rib_th + sp;

factory.addPoint(x1, height, 0, Lc1, 5);
factory.addPoint(x1, rib_th, 0, Lc1, 6);
factory.addPoint(x2, rib_th, 0, Lc1, 7);
factory.addPoint(x2, height, 0, Lc1, 8);

x1 = -width/2 + 2rib_th + sp;
x2 = -width/2 + 2rib_th + 2sp;

factory.addPoint(x1, height, 0, Lc1, 9);
factory.addPoint(x1, rib_th, 0, Lc1, 10);
factory.addPoint(x2, rib_th, 0, Lc1, 11);
factory.addPoint(x2, height, 0, Lc1, 12);


# for i in 1:(rib_num-1)
#     p1x = -width/2 + rib_th + (sp + rib_th) * (i - 1);
#     p2x = p1x;
#     p3x = p1x + sp;
#     p4x = p3x;

#     factory.addPoint(p1x, height)
# end

# обводка контура
factory.addLine(1, 2, 1);
factory.addLine(2, 3, 2);
factory.addLine(1, 4, 3);

for i in 4:11
    factory.addLine(i, i+1, i);
end
factory.addLine(12, 3, 12);

# объявление границ, по которым будут накладыватся граничные условия
factory.addPhysicalGroup(1, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], -1, "free");
factory.addPhysicalGroup(1, [1], -1, "fixed");

factory.synchronize();

# объединение контура
factory.addCurveLoop([-2, -1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 17);

# создание поверхности внутр контур
factory.addPlaneSurface([17], 18)


factory.synchronize()

# создание группы с поверхностью -> необходимо для генерации геометрии
factory.addPhysicalGroup(2, [18], -1, "coller");

factory.synchronize()


# генерация сетки
gmsh.model.mesh.generate(2)

# результат пишем в файл geo.msh в папке урока
name = "geo.msh" |> tcf;
print(name)
gmsh.write(name)

# заканчиваем работу gmsh
gmsh.finalize()


c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh

Полученная геометрия имеет две повехности - нижняя грань, по которой предполагается контакт с по верхностью, помечена меткой `"fixed"`, остальная часть границы помечена меткой `"free"`

# Загрузка геометрии в Gridap

Для загрузки .msh геометрии используется библиотека GridapGmsh, которая конвертирует в формат Gridap

In [16]:
using Gridap
using GridapGmsh

model = GmshDiscreteModel(name);

Info    : Reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh'...
Info    : 25 entities
Info    : 400 nodes
Info    : 798 elements
Info    : Done reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh'


Для просмотра геометрии удобнее всего использовать Paraview:

In [17]:
vtk_file = "model" |> tcf;
writevtk(model, vtk_file);

В результате появится 3 файла - `"model_0.vtu"`, `"model_1.vtu"` и `"model_2.vtu"`, содержащие вершины, ребра и грани. 
Можно легко проверить границы которые имеются в импортированном файле.

![image](free.png)
![image](fixed.png)

# Конечно элементное пространство

Как только мы получим дискретизацию рассчетной области, следующим шагом будет создание дискретной аппроксимации пространств конечных элементов $V_0$ и $U_g$ (т.е. КЭ-пространств тестовых и пробных функций) задачи. Для этого, во-первых, мы собираемся построить дискретизацию $V_0$ в качестве стандартного соответствующего лагранжева КЭ-пространства (с нулевыми граничными условиями), связанного с дискретизацией вычислительной области. Аппроксимация пространства FE $V_0$ строится следующим образом:

In [18]:
reffe = ReferenceFE(lagrangian,Float64,EL_ORDER)
V0 = TestFESpace(model,reffe;conformity=:H1);

Здесь мы использовали конструктор `TestFESpace`, который создает конкретное КЭ-пространство (для использования в качестве тестового пространства) из набора параметров, описанных как позиционные и именнованные аргументы. Первый позиционный аргумент - это модель, поверх которой мы хотим построить пространство. Второй позиционный аргумент содержит информацию о типе интерполяции FE (в данном случае эталонный FE). С помощью `ReferenceFE(lagrangian,Float64,EL_ORDER)` мы выбираем скалярно-значный лагранжианский эталонный FE порядка 1, где значение функций формы будет представлено 64-разрядными числами с плавающей запятой. С помощью аргумента соответствия ключевому слову мы определяем регулярность интерполяции на границах ячеек сетки. Здесь мы используем соответствие =:H1, что означает, что результирующее интерполяционное пространство является подмножеством $H^1 (\Omega)$ (т.е. непрерывных функций формы). С другой стороны, мы передаем идентификаторы границы Дирихле через аргумент `dirichlet_tags`. В этом случае мы помечаем как `Dirichlet` все объекты дискретной модели, идентифицированные тегом `"fixed"`. Поскольку это тестовое пространство, соответствующие функции формы обращаются в нуль на границе Дирихле.

Как только пространство $V_0$ дискретизировано, мы приступаем к аппроксимации пробного пространства $U_g$.

In [20]:
g(x) = 63
Ug = TrialFESpace(V0)

UnconstrainedFESpace()

С этой целью мы использовали конструкторы `TrialFESpace`. Обратите внимание, что при построении пробного пространства мы передали функцию, представляющую значение граничного условия Дирихле.

## Численное интегрирование



In [22]:
degree = EL_ORDER*2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [23]:
neumanntags = ["free"]
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

GenericMeasure()

## Численная процедура

Описание функционала, билинейной и линейной форм

In [24]:
T_a = 22;
h_t = 20;
λ = 210;

h(x) = h_t * T_a;
a(u,v) = ∫( λ * ∇(v)⋅∇(u) )*dΩ + ∫(h_t*u*v)*dΓ 
b(v) = ∫( v * h )*dΓ

b (generic function with 1 method)

In [25]:
op = AffineFEOperator(a,b,Ug,V0)

AffineFEOperator()

In [26]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [27]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 638
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 14035557112676587698

In [28]:
results_file = "results" |> tcf;
writevtk(Ω,results_file,cellfields=["uh"=>uh])

(["c:\\Users\\kutsj\\OneDrive\\Education\\Programming\\Julia\\CCM_tut\\lesson_1\\results.vtu"],)